In [3]:
import pandas as pd

path = "../../../../../Desktop/cv-cat-18/ca/validated.tsv"

In [4]:
df_full = pd.read_csv(path,delimiter="\t")

/var/folders/2d/7d3c_x1x5mb40mc1sgr14p740000gn/T/ipykernel_24944/2631694780.py:1: DtypeWarning: Columns (4,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(path,delimiter="\t")


In [7]:
import json

with open("./macrodial.json",'r',encoding='utf-8') as input: 
    macro_dial = json.load(input)

In [9]:
df_full["grouped_accents"] = df_full.apply(lambda row: macro_dial[row.accents] if macro_dial.get(row.accents, None) else "None",axis=1)

In [12]:
df_val = df_full[df_full.grouped_accents=='valencià']
df_cent = df_full[df_full.grouped_accents=='central']
df_nord = df_full[df_full.grouped_accents=='nord']
df_bal = df_full[df_full.grouped_accents=='balear']
df_no = df_full[df_full.grouped_accents=='nord-occidental']

In [21]:
val_eval = df_val.sample(n=12025)
cent_eval = df_cent.sample(n=12025)
nord_eval = df_nord.sample(n=12025)
bal_eval = df_bal.sample(n=12025)
no_eval = df_no.sample(n=12025)


In [22]:
df_full.columns

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment', 'grouped_accents'],
      dtype='object')

In [23]:
full_eval = pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval],ignore_index=True)

In [27]:
out_path = "../../../../../Desktop/cv-cat-18/ca/"
full_eval.to_csv(out_path+"eval_balanced.tsv",sep='\t')

In [28]:
len(df_full)

1885699

In [29]:
train_val_df = df_full.drop(pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval]).index)

In [30]:
len(train_val_df)

1825574

In [31]:
#Check that the lengths of the train_val set correspond to what they should after removing eval
assert (len(df_full)-len(train_val_df)) == len(full_eval)

In [32]:
pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval]).head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,grouped_accents
71485,46098bc705768ba56decc92e627fc907363d2e384241e8...,common_voice_ca_31318023.mp3,06f091c1ddc8c9da49f837a854a03ad8cdf3cfc2b6e8d0...,En el nivell superior de la façana lateral sud...,NaN,6,0,NaN,NaN,valencià,NaN,ca,NaN,valencià
83355,89a391019ba35cfbe3bd2a0324a8bcbdd7a6b670035d39...,common_voice_ca_35017249.mp3,3eddcd5f67cc86dc551a72894e214168bc7ef6deb1b8b9...,És llicenciat en dret i ciències econòmiques p...,NaN,2,0,fourties,male_masculine,valencià,NaN,ca,NaN,valencià
1509235,892bf89bd3a008a7d982de0d278349e654c713efacf965...,common_voice_ca_30680071.mp3,16828f0b9918d5e9ef08b6db980c345fdc1fff3e5d5303...,L'assassinat del president fou el detonant de ...,NaN,9,0,fourties,male_masculine,"valencià,La Vall d'Albaida",NaN,ca,NaN,valencià
1506720,892bf89bd3a008a7d982de0d278349e654c713efacf965...,common_voice_ca_30605614.mp3,6d08eaad1f5837fdf8b322ecdd1ffa6c3bbb6a84f17437...,La seva conclusió és que la pobresa és la caus...,NaN,6,0,fourties,male_masculine,"valencià,La Vall d'Albaida",NaN,ca,NaN,valencià
422905,fa525e3faa7dafd522be42b91c20a0c5364f1f45fbfe4c...,common_voice_ca_30741312.mp3,6d7139804857c9ba00136eccf6afc8a089124c24cf45c9...,Quin dia és el mercat d'Ademús?,NaN,4,0,twenties,male_masculine,Ebrenc,NaN,ca,NaN,valencià


In [47]:
print(71485 in list(train_val_df.index))

False


In [48]:
print(1 in list(train_val_df.index))

True


In [49]:
print(1506720 in list(train_val_df.index))

False


In [50]:
train_val_df.to_csv(out_path+"train_dev_full.tsv",sep='\t')